## Intro to Web Scraping

We've been collecting data using simple interfaces -- extensions to Chrome. As we mentioned, while open data portals, APIs and other publication mechanisms provide easy ways to get to information we need for our analysys and reporting, there are plenty other valuable data sources for us to take advantage of: web pages (HTML), PDF files, email dumps, etc. Automating the extraction of useful information from web pages is known as **"web scraping."** A terrible name aside, web scraping is very powerful and it's something you'll want to master. Today, we'll close our session talking about some of the basics of web scraping in Python.

![Web Scraping](https://blog.hartleybrody.com/wp-content/uploads/2012/12/scraper-tool.jpg)


There are many ways to scrape information from the web, but we're going to use Python, [requests](http://docs.python-requests.org/en/master/) and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/).

### Trump's Lies

Since we haven't talked about Trump nearly enough in this class(!), let's take a look at a New York Times piece on [all of the lies Trump told in 2017](https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html). Fun! This will lead to a good scraping exercise. Before we get to the code, take a quick look at the NYTimes piece.

The first part of web scraping is making HTTP requests to pull the pages we need. We will use the [requests](http://docs.python-requests.org/en/master/) library?

In [ ]:
# make the request to get the Trump Lies HTML
from requests import get

url = 'https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html'

headers = {
    'From': '<put your email here>',
}
response = get(url, headers=headers)

Above, we include a "header" field (represented as a dictionary). The header passes information to the web server that might change the way it returns content. In later exercises, we might need to specify the header "User-Agent" which tells the server what kind of  browser the requeste is being made from -- some servers don't like handing pages out to bots. 

For now, we are using the "From" header to announce ourselves. I like to tell a source that I'm taking data. If you want to know more about headers, have a look [here](https://en.wikipedia.org/wiki/List_of_HTTP_header_fields).

In [ ]:
# let's see what we have. remember that response.text will give us a string value of the page HTML

print(response.text)

This is kind of a mess. The whole web page has been read in as a string. Thankfully, one of the great things about Python is a package called [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/), designed by [Leonard Richardson](http://www.crummy.com/self/). It is truly a thing of beauty. BeautifulSoup is a parser for HTML (and XML) that creates an object that lets you interact with the components of a web page. You can search for tags, extract attributes from the tags and pull the content contained in a tag. [The documentation is pretty simple too.](http://www.crummy.com/software/BeautifulSoup/bs4/doc/) The latest version of BeautifulSoup is 4.6.0 and the package is called bs4.

We will come back to parsing Trump's lies with BeautifulSoup but let's start with a simple example first. Here is some very simple HTML that I'd like to run through BeautifulSoup:

```html
<html>

    <head>
        <title>My Technology News Site</title>
    </head>

    <body>
        <div>
            <p class="title"><strong>Steve Jobs introduces the public beta of Mac OS X</strong></p>
            <div class="description">Sept 13, 2000 - Steve Jobs <a href="https://www.apple.com/pr/library/2000/09/13Apple-Releases-Mac-OS-X-Public-Beta.html" target="_blank">introduces</a> the public beta of Mac OS X for US$29.95.</div>
            <div class="author">Author: Michael Young</div>
        </div>
    </body>

</html>

```

In [ ]:
from bs4 import BeautifulSoup

our_html = '''
<html>

    <head>
        <title>My Technology News Site
    </head>

    <body>
        <div>
            <p class="title"><strong>Steve Jobs introduces the public beta of Mac OS X</strong></p>
            <div class="description">Sept 13, 2000 - Steve Jobs <a href="https://www.apple.com/pr/library/2000/09/13Apple-Releases-Mac-OS-X-Public-Beta.html" target="_blank">introduces</a> the public beta of Mac OS X for US$29.95.</div>
            <div class="author">Author: Michael Young</div>
        </div>
    </body>

</html>
'''

# BeautifulSoup takes two arguments: a string (hopefully with HTML in it) and the parser we'd like to use
soup = BeautifulSoup(our_html, 'html.parser')

# print out a pretty version of the BeautifulSoup object
print(soup.prettify())

Let's do a super-quick review of [HTML](https://en.wikipedia.org/wiki/HTML):

Hypertext Markup Language is the language used for creating web pages. HTML uses `tags` which help label as well as structure the data in the document. Web browsers use the tags to help render the web page but does not display the tags. 

`Tags` normall come in pairs and have an opening tag `<p>` and a closing tag `</p>`:
```html
<p>this is a paragraph tag</p>
```

Other tags like the image tag `<img>` don't have a closing tag:
```html
<img src="http://somesite.com/images/logo.jpg" />
```

The other important thing about HTML tags is that they can contain one or more `attributes`. Like in the `<img>` tag above, the `src` attribute is used specify the URL of the image. A tag with multiple attributes could look like this:
```html
<p attribute_1="value1" attribute_2="value2">
Our content goes here
</p>
```

HTML documents typically have nested tags (think of a tree!) that looks like this:
```html
<html>
  <head>
    <title>My First Website!</title>
  </head>

    <body>
        <p>My mom would be proud of this.</p>
    </body>  
</html>
```



Back to BeautifulSoup...

When we run our HTML document through BeautifulSoup, we get a python object that allows us to traverse, query and manipulate the HTML document.

Here are a few ways to inspect our simple HTML document that we loaded above:

In [ ]:
# <title> tag
print(soup.title)

# name of the <title> tag
print(soup.title.name)

# string value in the <title> tag
print(soup.title.string)

In [ ]:
# how about if we want to find the first <div> tag?

soup.div

In [ ]:
# the string value for the first <p> tag within the first <div>

soup.div.p.string

In [ ]:
# the value of the "class" attribute of the first <div> under the first <div> (!?!)

soup.div.div['class']

In [ ]:
# here is how we'd find the the <a> tag

soup.div.div.a

In [ ]:
#### For You To Try

# how would you find the url in the description?



We can use `find` and `find_all` to search through the HTML to find certains tags and tag/attribute combinations. Let's take a look:

In [ ]:
# find all <p> tags
for p in soup.find_all('p'):
    print(p.text)

In [ ]:
# find all <div class="author>...</div> tags
for author_div in soup.find_all('div', attrs={'class': 'author'}):
    print(author_div.text)

### "The president is still lying..."

Coming back to the Trump's lies page, how can we use BeautifulSoup to parse our the lies and create a CSV data that we can use for our own analysis?


In [ ]:
from requests import get
from bs4 import BeautifulSoup

url = 'https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html'

headers = {
    'From': '<put your email here>'
}

# http request
response = get(url, headers=headers)

# run the HTML through BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# print out a pretty version of the BeautifulSoup object
print(soup.prettify())

Still a mess! Let's open up the [link](https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html) in Chrome and view the HTML there. Remember the `View Page Source` option that allows us to peek under the covers of any web page? Another great resource is [Chrome Developer Tools](https://developer.chrome.com/devtools) which gives you an even greater look under the hood! 

Do we see any patterns in the NYTimes HTML?

```html
<span class="short-desc"><strong>Jan. 21&nbsp;</strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>&nbsp;&nbsp;<span class="short-desc"><strong>Jan. 21&nbsp;</strong>“A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.” <span class="short-truth"><a href="http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/" target="_blank">(Trump was on the cover 11 times and Nixon appeared 55 times.)</a></span></span>&nbsp;&nbsp;
```

In [ ]:
# we can find each lie between the <span class="short-desc"> and </span> tags

for lie in soup.find_all('span', attrs={'class': 'short-desc'}):
    print(lie.prettify())

In [ ]:
# how would we find each date?

for lie in soup.find_all('span', attrs={'class': 'short-desc'}):
    date = lie.find('strong')
    print(date.string)

**There's another way!**

BeautifulSoup tags have a `contents` attribute returns a `list` of all of the tags children. The children in this case are all of the tags and strings nested under a tag.

In [ ]:
# print out the tag "contents"

for lie in soup.find_all('span', attrs={'class': 'short-desc'}):
    print(lie.contents)
    print('---\n')

In [ ]:
# another way to get the date

for lie in soup.find_all('span', attrs={'class': 'short-desc'}):
    date = lie.contents[0].string
    print(date)

In [ ]:
# how about getting the actual lie?

for lie in soup.find_all('span', attrs={'class': 'short-desc'}):
    lie_text = lie.contents[1].string
    print(lie_text)

**CSS Selectors**

We can also use CSS selectors with BeautifulSoup. Pairing SelectorGadget with BeautifulSoup is easy with the `.select()` method. You can copy your CSS selector directly as an argument. So give it a try! Here we pull just the dates of the lies. 

In [ ]:
soup.select(".short-desc strong")

Which we can now iterate over! Full circle!

### Putting It All Together

Now that we have the scraping part knocked out (congrats!), what if we wanted to save the data to a local `csv` file, or to pandas, to do further analysis? How might we do that?


In [ ]:
from csv import writer

output = writer(open("lie.csv","w"))
output.writerow(["date","description"])

for lie in soup.find_all('span', attrs={'class': 'short-desc'}):
    lie_text = lie.contents[1].string
    lie_date = lie.contents[0].string
    output.writerow([lie_date,lie_text])

&#x1f3c6; **Challenge round!** &#x1f3c6;

Pick one or two of these tasks and use your skills with web scraping to answer the question. In each case, there is a URL and a data question attached to it. These come mainly from an excellent list compiled by Dan Nguyen at Stanford.

>Site: [https://analytics.usa.gov/](https://analytics.usa.gov/)<br>
Task: Number of people visiting US Government web sites now<br><br>
Site: [http://www.state.gov/r/pa/ode/socialmedia/](http://www.state.gov/r/pa/ode/socialmedia/)<br>
Task: The number of Pinterest accounts maintained by U.S. State Department embassies and missions<br><br>
Site: [https://petitions.whitehouse.gov/](https://petitions.whitehouse.gov/)<br>
Task: Number of petitions that have reached their goal<br><br>
Site: [https://www.faa.gov/air_traffic/flight_info/aeronav/aero_data/](https://www.faa.gov/air_traffic/flight_info/aeronav/aero_data/)<br>
Task: Number of airports with existing construction related activity<br><br>
Site: [https://www.osha.gov/pls/imis/establishment.html](https://www.osha.gov/pls/imis/establishment.html)<br>
Number of OSHA enforcement inspections involving Wal-Mart in California since 2014<br><br>
Site: [https://www.tdcj.state.tx.us/death_row/dr_scheduled_executions.html](https://www.tdcj.state.tx.us/death_row/dr_scheduled_executions.html)<br>
Task: Number of days until Texas's next scheduled execution <br><br>